In [1]:
#
# XCoin API-call related functions
#
# @author	btckorea
# @date	2017-04-12
#
# Compatible with python3 version.

import sys
import time
import math
import base64
import hmac, hashlib
import urllib.parse
import pycurl
import json
import codecs

class XCoinAPI:
	api_url = "https://api.bithumb.com";
	api_key = "";
	api_secret = "";

	def __init__(self, api_key, api_secret):
		self.api_key = api_key;
		self.api_secret = api_secret;

	def body_callback(self, buf):
		self.contents = buf.decode('utf-8');


	def microtime(self, get_as_float = False):
		if get_as_float:
			return time.time()
		else:
			return '%f %d' % math.modf(time.time())

	def usecTime(self) :
		mt = self.microtime(False)
		mt_array = mt.split(" ")[:2];
		return mt_array[1] + mt_array[0][2:5];

	def xcoinApiCall(self, endpoint, rgParams):
		# 1. Api-Sign and Api-Nonce information generation.
		# 2. Request related information from the Bithumb API server.
		#
		# - nonce: it is an arbitrary number that may only be used once.
		# - api_sign: API signature information created in various combinations values.

		endpoint_item_array = {
			"endpoint" : endpoint
		};

		uri_array = dict(endpoint_item_array, **rgParams); # Concatenate the two arrays.

		str_data = urllib.parse.urlencode(uri_array);

		nonce = self.usecTime();

		data = endpoint + chr(0) + str_data + chr(0) + nonce;
		utf8_data = data.encode('utf-8');

		key = self.api_secret;
		utf8_key = key.encode('utf-8');

		h = hmac.new(bytes(utf8_key), utf8_data, hashlib.sha512);
		hex_output = h.hexdigest();
		utf8_hex_output = hex_output.encode('utf-8');

		api_sign = base64.b64encode(utf8_hex_output);
		utf8_api_sign = api_sign.decode('utf-8');

		curl_handle = pycurl.Curl();
		curl_handle.setopt(pycurl.POST, 1);
		curl_handle.setopt(pycurl.VERBOSE, 1); # vervose mode :: 1 => True, 0 => False
		curl_handle.setopt(pycurl.POSTFIELDS, str_data);

		url = self.api_url + endpoint;
		curl_handle.setopt(curl_handle.URL, url);
		curl_handle.setopt(curl_handle.HTTPHEADER, ['Api-Key: ' + self.api_key, 'Api-Sign: ' + utf8_api_sign, 'Api-Nonce: ' + nonce]);
		curl_handle.setopt(curl_handle.WRITEFUNCTION, self.body_callback);

		curl_handle.perform()

		response_code = curl_handle.getinfo(pycurl.RESPONSE_CODE); # Get http response status code.
		curl_handle.close();
		#print("xx==xx")
		#print(self.contents)
		return (json.loads(self.contents));
		#return (self.contents);


In [5]:
#! /usr/bin/env python
# XCoin API-call sample script (for Python 3.X)
#
# @author	btckorea
# @date	2017-04-11
#
#
# First, Build and install pycurl with the following commands::
# (if necessary, become root)
#
# https://pypi.python.org/pypi/pycurl/7.43.0#downloads
#
# tar xvfz pycurl-7.43.0.tar.gz
# cd pycurl-7.43.0
# python setup.py --libcurl-dll=libcurl.so install
# python setup.py --with-openssl install
# python setup.py install

import sys
#from xcoin_api_client import *
import pprint


api_key = "xxxx";
api_secret = "xxxxx";

api = XCoinAPI(api_key, api_secret);
# * {currency} = BTC, ETH, DASH, LTC, ETC, XRP, BCH, XMR, ZEC, QTUM, BTG, EOS (基本值: BTC), ALL(全部)
rgParams = {
	"order_currency" : "BTC",
	"payment_currency" : "KRW",
	"currency" : "ETH"
};


#
# public api
#
# /public/ticker
# /public/recent_ticker
# /public/orderbook
# /public/recent_transactions


#ticker
print("ticker")
result = api.xcoinApiCall("/public/ticker", rgParams);
print( result)

#recent_transactions
print("recent_transactions")
result = api.xcoinApiCall("/public/recent_transactions", rgParams);
print( result)

'''
print("status: " + result["status"]);
print("buy: " + result["data"]["buy_price"]);
'''


#recent_ticker
print("recent_ticker")
recent_ticker = api.xcoinApiCall("/public/recent_ticker", rgParams);
print(recent_ticker)


#orderbook
print("orderbook")
orderbook = api.xcoinApiCall("/public/orderbook", rgParams);
print( orderbook)

#
# private api
#
# endpoint		=> parameters
# /info/current
# /info/account
# /info/balance
# /info/wallet_address

print("account: " )
result = api.xcoinApiCall("/info/account", rgParams);
print(result)

print("blance:")
result = api.xcoinApiCall("/info/balance", rgParams);
print(result)

print("wallet:")
result = api.xcoinApiCall("/info/wallet_address", rgParams);
print(result)

print("ticker:")
result = api.xcoinApiCall("/info/ticker", rgParams);
print(result)

print("orders:")
result = api.xcoinApiCall("/info/orders", rgParams);
print(result)

print("user_transactions:")
result = api.xcoinApiCall("/info/user_transactions", rgParams);
print(result)


print("place:")
result = api.xcoinApiCall("/trade/place", rgParams);
print(result)



sys.exit(0);



ticker
{'status': '0000', 'data': {'buy_price': '18984000', 'closing_price': '18976000', 'volume_7day': '175712.22230550', 'units_traded': '18826.09761906', 'opening_price': '19485000', 'sell_price': '18988000', 'min_price': '18669000', 'volume_1day': '18826.09761906', 'max_price': '19730000', 'date': '1514558741125', 'average_price': '19097798.0127'}}
recent_transactions
{'status': '0000', 'data': [{'type': 'ask', 'price': '18984000', 'total': '3086798', 'units_traded': '0.1626', 'transaction_date': '2017-12-29 23:45:38'}, {'type': 'bid', 'price': '18984000', 'total': '180348', 'units_traded': '0.0095', 'transaction_date': '2017-12-29 23:45:36'}, {'type': 'bid', 'price': '18984000', 'total': '68342', 'units_traded': '0.0036', 'transaction_date': '2017-12-29 23:45:36'}, {'type': 'bid', 'price': '18984000', 'total': '324626', 'units_traded': '0.0171', 'transaction_date': '2017-12-29 23:45:36'}, {'type': 'ask', 'price': '18976000', 'total': '315002', 'units_traded': '0.0166', 'transactio

SystemExit: 0

C:\ProgramData\Anaconda2\envs\py3\lib\site-packages\IPython\core\interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
%tb


SystemExit: 0